In [1]:
import ucimlrepo
from ucimlrepo import fetch_ucirepo

# Decision Trees

This notebook will explore the effectiveness of decision trees and random forests.


In [2]:
bank_marketing = fetch_ucirepo(id=222)
x = bank_marketing.data.features
y = bank_marketing.data.targets

In [3]:
for title in bank_marketing.metadata:
    if type(bank_marketing.metadata[title]) == ucimlrepo.dotdict:
        print(f"{title}:")
        for inner_title in bank_marketing.metadata[title]:
            print(f"\t{inner_title}: {bank_marketing.metadata[title][inner_title]}")
    else:
        print(f"{title}: {bank_marketing.metadata[title]}")

uci_id: 222
name: Bank Marketing
repository_url: https://archive.ics.uci.edu/dataset/222/bank+marketing
data_url: https://archive.ics.uci.edu/static/public/222/data.csv
abstract: The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe a term deposit (variable y).
area: Business
tasks: ['Classification']
characteristics: ['Multivariate']
num_instances: 45211
num_features: 16
feature_types: ['Categorical', 'Integer']
demographics: ['Age', 'Occupation', 'Marital Status', 'Education Level']
target_col: ['y']
index_col: None
has_missing_values: yes
missing_values_symbol: NaN
year_of_dataset_creation: 2014
last_updated: Fri Aug 18 2023
dataset_doi: 10.24432/C5K306
creators: ['S. Moro', 'P. Rita', 'P. Cortez']
intro_paper:
	ID: 277
	type: NATIVE
	title: A data-driven approach to predict the success of bank telemarketing
	authors: Sérgio Moro, P. Cortez, P. Rita
	venue: Decision Su

In [4]:
bank_marketing.variables

,name,role,type,demographic,description,units,missing_values
0,age,Feature,Integer,Age,None,None,no
1,job,Feature,Categorical,Occupation,"type of job (categorical: 'admin.','blue-colla...",None,no
2,marital,Feature,Categorical,Marital Status,"marital status (categorical: 'divorced','marri...",None,no
3,education,Feature,Categorical,Education Level,"(categorical: 'basic.4y','basic.6y','basic.9y'...",None,no
4,default,Feature,Binary,None,has credit in default?,None,no
5,balance,Feature,Integer,None,average yearly balance,euros,no
6,housing,Feature,Binary,None,has housing loan?,None,no
7,loan,Feature,Binary,None,has personal loan?,None,no
8,contact,Feature,Categorical,None,contact communication type (categorical: 'cell...,None,yes
9,day_of_week,Feature,Date,None,last contact day of the week,None,no


In [5]:
x

,age,job,marital,education,default,balance,housing,loan,contact,day_of_week,month,duration,campaign,pdays,previous,poutcome
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,may,261,1,-1,0,NaN
1,44,technician,single,secondary,no,29,yes,no,NaN,5,may,151,1,-1,0,NaN
2,33,entrepreneur,married,secondary,no,2,yes,yes,NaN,5,may,76,1,-1,0,NaN
3,47,blue-collar,married,NaN,no,1506,yes,no,NaN,5,may,92,1,-1,0,NaN
4,33,NaN,single,NaN,no,1,no,no,NaN,5,may,198,1,-1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,NaN
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,NaN
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,NaN


In [6]:
y

,y
0,no
1,no
2,no
3,no
4,no
...,...
45206,yes
45207,yes
45208,yes
45209,no
